<a href="https://colab.research.google.com/github/CidClayQuirino/rnn-component-lIfe-cycle/blob/main/TccUsp_SentimentAnalysis_Testep_Valor%2006_04-2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import pandas as pd

# URL do arquivo CSV no GitHub
url = "https://raw.githubusercontent.com/CidClayQuirino/rnn-component-lIfe-cycle/main/DadosVar200.csv"

# Importar o DataFrame do GitHub
df = pd.read_csv(url)

# Crie um novo DataFrame com apenas as colunas desejadas
novo_df = df.loc[:, ["data", "Variacao_Brent", "Variacao_Petrobras"]]

# Se você quiser modificar o DataFrame original, pode fazer assim:
# df = df.loc[:, ["data", "Variacao_Brent", "Variacao_Petrobras"]]

# Exibindo o novo DataFrame
print(novo_df)


           data  Variacao_Brent  Variacao_Petrobras
0    2023-06-12        3.410363           -5.078677
1    2023-06-13       -1.467228            4.672192
2    2023-06-14        3.374319           -0.719945
3    2023-06-15        1.242239            0.797677
4    2023-06-16       -0.926771            2.374100
..          ...             ...                 ...
195  2024-03-21       -0.408019            0.410113
196  2024-03-22        1.545124            1.701838
197  2024-03-25       -0.576369           -1.338687
198  2024-03-26       -0.185511            1.289013
199  2024-03-27        1.614598            1.875417

[200 rows x 3 columns]


In [76]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Supondo que 'novo_df' seja o DataFrame com colunas (data, Variacao_Brent, Variacao_Petrobras)

# Selecionar apenas as colunas de interesse para normalização
df_selected = novo_df[['data', 'Variacao_Brent', 'Variacao_Petrobras']]

# Normalizar os dados
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_selected[['Variacao_Brent', 'Variacao_Petrobras']]), columns=['Variacao_Brent', 'Variacao_Petrobras'])

# Adicionar a coluna de data ao DataFrame normalizado
df_normalized['data'] = novo_df['data']

# Criar uma lista para armazenar os resultados das correlações
correlacoes = []

# Iterar sobre o DataFrame normalizado em blocos de 3 linhas
for i in range(0, len(df_normalized), 3):
    # Selecionar o bloco de 3 linhas
    bloco = df_normalized.iloc[i:i+3]

    # Calcular a correlação entre as colunas normalizadas 'Variacao_Brent' e 'Variacao_Petrobras' para o bloco atual
    correlacao = bloco['Variacao_Brent'].corr(bloco['Variacao_Petrobras'])

    # Armazenar o resultado da correlação na lista
    correlacoes.append(correlacao)

# Criar um novo DataFrame com os resultados das correlações, a coluna de data e os dados originais
df_correlacoes = pd.DataFrame({
    'Data': novo_df['data'].iloc[::3].values,  # Selecionar a data do primeiro elemento de cada bloco
    'Correlacao_Variacao_Brent_Petrobras': correlacoes,
    'Variacao_Brent': df_selected['Variacao_Brent'].iloc[::3].values,
    'Variacao_Petrobras': df_selected['Variacao_Petrobras'].iloc[::3].values
})

# Exibir o novo DataFrame com as correlações, a coluna de data e os dados originais
df_correlacoes.head(100)


,Data,Correlacao_Variacao_Brent_Petrobras,Variacao_Brent,Variacao_Petrobras
0,2023-06-12,-0.897793,3.410363,-5.078677
1,2023-06-15,0.272318,1.242239,0.797677
2,2023-06-21,0.339712,-3.864112,-1.740290
3,2023-06-26,0.455699,-2.588296,-1.248268
4,2023-06-29,-0.158949,0.753303,-3.489184
...,...,...,...,...
62,2024-03-08,-0.640435,0.158379,-1.488493
63,2024-03-13,0.841714,1.654170,-0.607699
64,2024-03-18,-0.775195,0.563929,-0.411243
65,2024-03-21,0.858360,-0.408019,0.410113


Modelo de regressão linear Multipla

Os resultados das métricas de erro são os seguintes:

Mean Squared Error (MSE): 0.2419
Mean Relative Error (MRE): 2.1168
R-squared (R2): 0.4808
Isso significa que:

O MSE indica a média dos quadrados dos erros entre as previsões do modelo e os valores reais. Neste caso, um valor de MSE de 0.2419 sugere que, em média, o modelo está cometendo erros significativos em suas previsões.
O MRE indica a média dos erros relativos, que é a razão entre o erro absoluto e o valor médio dos dados reais. Um MRE de 2.1168 sugere que, em média, o modelo está cometendo erros relativamente grandes em relação ao valor médio dos dados reais.
O R2 (coeficiente de determinação) varia entre 0 e 1 e indica a proporção da variância na variável dependente que é previsível a partir das variáveis independentes. Um R2 de 0.4808 sugere que aproximadamente 48.08% da variabilidade na variável dependente é explicada pelas variáveis independentes no modelo. Isso indica que o modelo tem uma capacidade moderada de explicar a variabilidade nos dados.

In [85]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Supondo que você já tenha um DataFrame df com as colunas mencionadas
df = pd.DataFrame(df_correlacoes)

# Definindo as features e o target
features = ['Variacao_Brent', 'Variacao_Petrobras']
target = 'Correlacao_Variacao_Brent_Petrobras'

X = df[features]
y = df[target]

# Adicionando a constante para a regressão linear
X = sm.add_constant(X)

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando e treinando o modelo de regressão linear usando statsmodels
model = sm.OLS(y_train, X_train).fit()

# Prevendo os valores do conjunto de teste
y_pred = model.predict(X_test)

# Calculando as métricas de erro
mse = mean_squared_error(y_test, y_pred)
mre = mean_absolute_error(y_test, y_pred) / np.mean(y_test)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Relative Error (MRE):", mre)
print("R-squared (R2):", r2)


Mean Squared Error (MSE): 0.2418991150985183
Mean Relative Error (MRE): 2.1167742419706173
R-squared (R2): 0.4807649906701834


In [86]:
import pandas as pd

# URL do arquivo CSV no GitHub
url = "https://raw.githubusercontent.com/CidClayQuirino/rnn-component-lIfe-cycle/main/DadosVar200.csv"

# Importar o DataFrame do GitHub
df = pd.read_csv(url)

# Crie um novo DataFrame com apenas as colunas desejadas
# novo_df = df.loc[:, ["data", "Variacao_Brent", "Variacao_Petrobras"]]

# Se você quiser modificar o DataFrame original, pode fazer assim:
# df = df.loc[:, ["data", "Variacao_Brent", "Variacao_Petrobras"]]

# Exibindo o novo DataFrame
print(novo_df)


           data  Variacao_Brent  Variacao_Petrobras
0    2023-06-12        3.410363           -5.078677
1    2023-06-13       -1.467228            4.672192
2    2023-06-14        3.374319           -0.719945
3    2023-06-15        1.242239            0.797677
4    2023-06-16       -0.926771            2.374100
..          ...             ...                 ...
195  2024-03-21       -0.408019            0.410113
196  2024-03-22        1.545124            1.701838
197  2024-03-25       -0.576369           -1.338687
198  2024-03-26       -0.185511            1.289013
199  2024-03-27        1.614598            1.875417

[200 rows x 3 columns]


In [ ]:
!pip install statsmodels
import pandas as pd
from statsmodels.tsa.stattools import adfuller

In [ ]:
# URL do arquivo CSV no GitHub
url = "https://github.com/CidClayQuirino/rnn-component-lIfe-cycle/raw/main/valoresPBR.csv"

# Importar o DataFrame do arquivo CSV
df = pd.read_csv(url)

# Exibir as primeiras linhas do DataFrame para verificar se foi importado corretamente
print(df.head())

In [ ]:
df = pd.DataFrame(df)

# Converter a coluna 'Data' para o tipo datetime
df['data'] = pd.to_datetime(df['data'])

# Definir o índice como a coluna de datas
df.set_index('data', inplace=True)

# Aplicar o teste Dickey-Fuller aumentado
adf_result = adfuller(df['pbr'])

# Exibir o resultado do teste
print('Estatística ADF:', adf_result[0])
print('Valor-p:', adf_result[1])
print('Valores Críticos:')
for key, value in adf_result[4].items():
    print(f'\t{key}: {value}')
